In [1]:
from PIL import Image
file_path = 'input1.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()
    width, height, _ = map(int, lines[0].split())
    pixels = []
    for line in lines[1:]:
        pixel_data = line.replace('(', '').replace(')', ',').replace('\n', '').split(', ')
        pixels.extend((int(pixel_data[i]), int(pixel_data[i+1]), int(pixel_data[i+2])) for i in range(0, len(pixel_data) -1, 3))
image = Image.new("RGB", ( height, width))
image.putdata(pixels)
output_path = 'output_image.png'
image.save(output_path)
print(f"Image saved to {output_path}")

Image saved to output_image.png


In [2]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from skimage import exposure

image = cv2.imread('output_image.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

equalized = exposure.equalize_adapthist(gray, clip_limit=0.03)
equalized = (equalized * 255).astype(np.uint8)
_, thresholded = cv2.threshold(equalized, 150, 255, cv2.THRESH_BINARY)
kernel = np.ones((3, 3), np.uint8)
cleaned = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)
min_star_size = 10
contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
mask = np.zeros_like(cleaned)
for contour in contours:
    if cv2.contourArea(contour) >= min_star_size:
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
cv2.imwrite('filtered_stars_mask.jpg', mask)


True

In [3]:
import cv2
import numpy as np
from scipy.ndimage import label, find_objects
mask = cv2.imread('filtered_stars_mask.jpg', cv2.IMREAD_GRAYSCALE)
_, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
labeled_array, num_features = label(binary_mask)
print("Number of stars detected:", num_features)
star_centers = []
for i in range(1, num_features + 1):
    slices = find_objects(labeled_array == i)
    if slices:
        y_slice, x_slice = slices[0]
        cX = int((x_slice.start + x_slice.stop - 1) / 2)
        cY = int((y_slice.start + y_slice.stop - 1) / 2)
        star_centers.append((cX, cY))
print("Coordinates of stars:")
for i, (x, y) in enumerate(star_centers):
    print(f"Star {i+1}: ({x}, {y})")

Number of stars detected: 11
Coordinates of stars:
Star 1: (90, 131)
Star 2: (147, 135)
Star 3: (155, 143)
Star 4: (157, 153)
Star 5: (57, 171)
Star 6: (118, 206)
Star 7: (22, 216)
Star 8: (154, 218)
Star 9: (156, 260)
Star 10: (233, 271)
Star 11: (225, 297)
